In [1]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
from datetime import datetime
import json 

In [2]:
def read_data(path):
    df = pd.read_csv(path)
    del df["Unnamed: 0"]
    df['time'] = pd.to_datetime(df['time'])
    return df

In [3]:
def get_one(x):
    y=0
    if x[4]!=0:
        y=2
    elif x[1]!=0:
        y=1
    return y

In [4]:
class recys_process(object):
    def __init__(self,df):
        self.df = df
        self.unique_user_id = df.drop_duplicates("user_id")[["user_id","item_id"]]
        self.unique_category = df.drop_duplicates("item_category")[["item_category","item_id"]]
        self.unique_item = df.drop_duplicates("item_id")[["item_category","item_id"]]
    
    def time_per(self,day,per_d = 1,h1=0,h2=0):
        time_per = 0
        if per_d == 0:
            if day <= 13:
                time_per = self.df[(df.time >= datetime(2014,11,day+17,h1)) & (df.time < datetime(2014,11,day + per_d + 17,h2))]
            else:
                time_per = self.df[(df.time >= datetime(2014,12,day-13,h1)) & (df.time < datetime(2014,12,day + per_d -13,h2))]
        else:      
            if day < 13 :
                if day + per_d + 17 <= 30:
                    time_per = self.df[(df.time >= datetime(2014,11,day+17,h1)) & (df.time < datetime(2014,11,day + per_d + 17,h2))]
                else:
                    time_per = self.df[(df.time >= datetime(2014,11,day+17,h1)) & (df.time < datetime(2014,12,day + per_d + 17 - 30,h2))]
            if day == 13:
                    time_per = self.df[(df.time >= datetime(2014,11,day+17,h1)) & (df.time < datetime(2014,12,per_d,h2))]
            if day > 13:
                time_per = self.df[(df.time >= datetime(2014,12,day-13,h1)) & (df.time < datetime(2014,12,day + per_d -13,h2))]
        return time_per
    
    def exe_group(self,keys,days,*args):
        df = self.time_per(days,*args)
        df = df.groupby(keys).count().unstack().fillna(0).astype(int).item_id
        return df 
    
    def user_basic(self,keys,days,*args):
        num_category = self.time_per(days,*args).drop_duplicates(["user_id","item_category"]).groupby("user_id").count()
        num_item = self.time_per(days,*args).drop_duplicates(["user_id","item_id"]).groupby("user_id").count()
        user_basic = self.exe_group(keys,days,*args)
        user_basic.columns = ['click','collect','cart','buy']
        user_basic.insert(0,'user_id',user_basic.index)
        user_basic.insert(1,'num_category',num_category.item_category)
        user_basic.insert(2,'num_item',num_item.item_id)
        user_basic = pd.merge(self.unique_user_id,user_basic,how="outer",on="user_id").drop("item_id",axis=1).fillna(0).astype(int)
        return user_basic
    
    def category_basic(self,keys,days,*args):
        num_users = self.time_per(days,*args).drop_duplicates(["user_id","item_category"]).groupby("item_category").count()
        num_items = self.time_per(days,*args).drop_duplicates(["item_id","item_category"]).groupby("item_category").count()
        category_basic = self.exe_group(keys,days,*args)
        category_basic.columns = ['click','collect','cart','buy']
        category_basic.insert(0,"item_category",category_basic.index)
        category_basic.insert(1,"num_users",num_users.user_id)
        category_basic.insert(2,"num_items",num_items.item_id)
        category_basic = pd.merge(self.unique_category,category_basic,how="outer",on="item_category").drop("item_id",axis=1).fillna(0).astype(int)
        return category_basic
    
    def item_basic(self,keys,days,*args):
        num_users = self.time_per(days,*args).drop_duplicates(["user_id","item_id"]).groupby("item_id").count()
        item_basic = self.exe_group(keys,days,*args)
        item_basic.columns = ['click','collect','cart','buy']
        item_basic.insert(0,"item_id",item_basic.index)
        item_basic.insert(1,"num_users",num_users.user_id)
        item_basic = pd.merge(self.unique_item,item_basic,how="outer",on="item_id").fillna(0).astype(int)
        return item_basic
    
    def user_30days(self,day=0,*group_keys):
        if group_keys:
            keys = list(group_keys)
        keys = [df.user_id,df.behavior_type]
        basic_days = 0
        if day==0:
            basic_days = []
            for i in range(1,31,1):
                basic_days.append(self.exe_group(keys,i).apply(get_one,axis=1))
            basic_days = pd.concat(basic_days,axis=1).fillna(0).astype(int)
        else:
            basic_days = self.exe_group(keys,day).apply(get_one,axis=1)
        return basic_days
        

In [5]:
# test cmds
# recs.user_30days().ix[:,23:27].head(10)

# recs.exe_group(group_keys,3,2).ix[263670]

# df_3 = recs.time_per(3,2)
# df_3[df_3.user_id==263670]

In [6]:
if __name__=="__main__":
#     path = "E:\\projects\\recys_projects\\dataset\\0_999_user_info.csv"
#     path = "/home/xuyayuan/recys_projects/dataset/0_999_user_info.csv"
    path = "/home/projects/recys/recys_projects/dataset/0_999_user_info."
    df = read_data(path)
    group_keys = [df.user_id,df.item_category,df.item_id,df.behavior_type]
    group_keys_1 = [df.user_id,df.behavior_type]
    group_keys_2 = [df.user_id,df.item_category]
    group_keys_3 = [df.item_category,df.behavior_type]
    group_keys_4 = [df.item_id,df.behavior_type]
    recs = recys_process(df)
    df_days = recs.user_30days()

FileNotFoundError: File b'E:\\projects\\recys_projects\\dataset\\0_999_user_info.csv' does not exist

In [ ]:
user_basic = recs.user_basic(group_keys_1,1,28)
category_basic = recs.category_basic(group_keys_3,1,28)
item_basic = recs.item_basic(group_keys_4,1,28)

In [ ]:
category_basic

In [ ]:
user_basic

In [ ]:
item_basic